In [43]:
import h5py
import numpy as np
from mc_lib.observable import RealObservable
import utils

In [164]:
def save_RO_array_in_group(group, arr):
    Z_b = group.create_dataset('Z_b', shape=len(arr))
    for i, ro in enumerate(arr):
        blocks, z = ro.__getstate__()
        group.create_dataset(f'RO_blocks_{i}', data=blocks)
        Z_b[i] = z

        
def load_RO_array_from_group(group):
    n = len(group.keys())-1
    arr = np.empty(n, dtype=RealObservable)
    for i in range(n):
        a = RealObservable()
        a.__setstate__((group[f'RO_blocks_{i}'][()], group['Z_b'][i]))
        arr[i] = a
    
    return arr
        
def save_conf_hdf5(conf, fname):
    with h5py.File(fname, "w") as f:
        f.create_dataset('structure', data=conf.struct)
        f.create_dataset('betas', data=conf.betas)
        
        mag2 = f.create_group('mag2')
        save_RO_array_in_group(mag2, conf.mag2)
        
        mag4 = f.create_group('mag4')
        save_RO_array_in_group(mag4, conf.mag4)
        
        ene = f.create_group('ene')
        save_RO_array_in_group(ene, conf.ene)

In [149]:
c = utils.Conformation()

In [150]:
c.load_data('Conformations/L1000_beta0.1_1_10(10000)/conf_data_0.npz')
c.load_struct('Conformations/L1000_beta0.1_1_10(10000)/struct_conf_0.dat')

In [153]:
save_conf_hdf5(c, 'Conformations/Tests/conf.hdf5')

In [174]:
f = h5py.File('Conformations/Tests/conf.hdf5', "r")

In [175]:
m = f['mag2']
len(m.keys())

11

In [177]:
mag2 = load_RO_array_from_group(m)
mag2[-1].mean

0.22741689890898104

In [134]:
f.get('structure')[()]

array([[ 0,  0],
       [ 0,  1],
       [-1,  1],
       ...,
       [ 8, 80],
       [ 7, 80],
       [ 7, 81]])

In [135]:
m = f.get('mag2')

In [136]:
m.keys()

<KeysViewHDF5 ['RO_blocks_0', 'RO_blocks_1', 'RO_blocks_2', 'RO_blocks_3', 'RO_blocks_4', 'RO_blocks_5', 'RO_blocks_6', 'RO_blocks_7', 'RO_blocks_8', 'RO_blocks_9', 'Z_b']>

In [137]:
m.get('Z_b')[()]

array([512., 512., 512., 512., 512., 512., 512., 512., 512., 512.],
      dtype=float32)

In [138]:
b = RealObservable()
print(b.mean)
b.__setstate__((m.get('RO_blocks_9')[()], m.get('Z_b')[9]))

0.0


In [139]:
b.mean

0.22741689890898104

In [140]:
c.mag2[-1].mean

0.22741689890898104

In [141]:
c.betas

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [142]:
f['betas'][()]

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [178]:
f.close()